In [1]:
import SimpleITK as sitk

def save_array_to_nii(mask_array, nii_template_path, nii_save_path):

    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()
#     print(output_spacing ,output_direction,output_origin)

    nrrd_output = sitk.GetImageFromArray(mask_array)
    nrrd_output.SetSpacing(output_spacing)
    nrrd_output.SetDirection(output_direction)
    nrrd_output.SetOrigin(output_origin)

    nrrdWriter = sitk.ImageFileWriter()
    nrrdWriter.SetFileName(nii_save_path)
    nrrdWriter.SetUseCompression(True)
    nrrdWriter.Execute(nrrd_output)
    print(nii_save_path ,'saved')

In [2]:
import os
data_dir = 'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/'
image_paths = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if '.nii.gz' in file :
                img_path = os.path.join(r, file)
                image_paths.append(img_path)

image_paths = sorted(image_paths)
print(len(image_paths))

159


In [3]:
# import numpy as np
# from skimage.measure import label   

# def getLargestCC(prd_array):
    
#     prd_array =  prd_array.astype(int)
#     for i in range(prd_array.shape[0]):
#         prd_slice = prd_array[i,:,:]
#         labels = label(prd_slice)
#         assert( labels.max() != 0 ) # assume at least 1 CC
#         largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
#         prd_array[i,:,:] = largestCC
#         print(i)
#     return prd_array.astype(int)

In [6]:
import shutil,random
import SimpleITK as sitk  
from test_scripts.image_postprocessing_test import get_biggst_mask_array
from test_scripts.sitk_nifti_tools import get_nifti_from_array_via_interspacing,nifiti_save_into_path

import numpy as np

targ_dir = 'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/'

for idx,img_path in enumerate(image_paths):
    print(idx, img_path)
    seg_path = img_path
    d = img_path.split('/')[-1]
    targ_path = targ_dir + d.replace('.nii.gz','_largest_curated.nii.gz')
    
    template_sitk =sitk.ReadImage(seg_path)
    prdCube =  sitk.GetArrayFromImage(template_sitk)

    prdCube1 = np.copy(prdCube)
    prdCube1[prdCube1!=1]=0
    prdCube1[prdCube1!=0]=1
    prdCube1 = get_biggst_mask_array(prdCube1)
    prdCube1 = prdCube1 * 1

    prdCube2 = np.copy(prdCube)
    prdCube2[prdCube2!=2]=0
    prdCube2[prdCube2!=0]=1
    prdCube2 = get_biggst_mask_array(prdCube2)
    prdCube2 = prdCube2 * 2
    
    prdCube3 = np.copy(prdCube)
    prdCube3[prdCube3!=3]=0
    prdCube3[prdCube3!=0]=1
    prdCube3 = get_biggst_mask_array(prdCube3)
    prdCube3 = prdCube3 * 3
    
    prdCube4 = np.copy(prdCube)
    prdCube4[prdCube4!=4]=0
    prdCube4[prdCube4!=0]=1
    prdCube4 = get_biggst_mask_array(prdCube4)
    prdCube4 = prdCube4 * 4
    
    print(prdCube4.shape)
    
    seg_array =  prdCube1 +  prdCube2 +  prdCube3 +  prdCube4
#     seg_array =   prdCube4

    print(seg_array.shape)

#     inter_spacing = template_sitk.GetSpacing()
#     sitk_output = get_nifti_from_array_via_interspacing(seg_array,inter_spacing,template_sitk)
#     nifiti_save_into_path(sitk_output,targ_path)
    
#     seg_array =  prdCube1 +  prdCube2 +  prdCube3 +  prdCube4
    save_array_to_nii(seg_array, seg_path, targ_path )
    print(idx,'th curated and saved',targ_path)    
    print(' \n\n\n')

0 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/10_T1InPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 485, 485)
(8, 485, 485)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/10_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
0 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/10_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



1 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/10_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/10_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
1 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/10_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



2 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/10_T2SPIR_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/

(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/16_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
24 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/16_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



25 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/16_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/16_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
25 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/16_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



26 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/16_T2SPIR_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/16_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
26 th curated and sav

(8, 320, 320)
(8, 320, 320)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/20_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
46 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/20_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



47 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/21_T1DUAL_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/21_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
47 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/21_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



48 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/21_T1InPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/21_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
48 th curated and saved nnUNet_ra

(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/26_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
68 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/26_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



69 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/26_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/26_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
69 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/26_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



70 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/26_T2SPIR_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/26_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
70 th curated and sav

(8, 400, 400)
(8, 400, 400)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/30_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
88 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/30_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



89 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/30_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 400, 400)
(8, 400, 400)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/30_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
89 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/30_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



90 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/30_T2SPIR_img_crop8slices_nnunet_to_seg.nii.gz
(8, 512, 512)
(8, 512, 512)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/30_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
90 th curated and sav

(8, 384, 384)
(8, 384, 384)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/35_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
108 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/35_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



109 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/35_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 384, 384)
(8, 384, 384)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/35_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
109 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/35_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



110 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/35_T2SPIR_img_crop8slices_nnunet_to_seg.nii.gz
(8, 512, 512)
(8, 512, 512)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/35_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
110 th curated an

nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/3_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
130 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/3_T2SPIR_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



131 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/40_T1DUAL_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/40_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
131 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/40_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



132 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/40_T1InPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 256, 256)
(8, 256, 256)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/40_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
132 th curated and saved nnUNet_raw_data_base/nnUNet_test_d

(8, 288, 288)
(8, 288, 288)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/8_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
151 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/8_T1DUAL_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



152 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/8_T1InPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 288, 288)
(8, 288, 288)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/8_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
152 th curated and saved nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/8_T1InPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz
 



153 nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/8_T1OutPhase_img_crop8slices_nnunet_to_seg.nii.gz
(8, 288, 288)
(8, 288, 288)
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/8_T1OutPhase_img_crop8slices_nnunet_to_seg_largest_curated.nii.gz saved
153 th curated and saved 

In [5]:
for i in range(seg_array.shape[0]):
    print(i)

0
1
2
3
4
5
6
7
